# TensorBoard Basics

In [10]:
# Mrityunjay kumar #
#More stuff here: https://blog.jakuba.net/2017/05/30/tensorflow-visualization.html

In [11]:
import input_data
import tensorflow as tf


tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()


mnist = input_data.read_data_sets("MNIST/", one_hot=True)
# Set parameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [12]:
# TF graph input
x = tf.compat.v1.placeholder("float", [None, 784]) # mnist data image of shape 28*28=784
y = tf.compat.v1.placeholder("float", [None, 10]) # 0-9 digits recognition => 10 classest

In [13]:
# Create a model

# Set model weights
W = tf.compat.v1.Variable(tf.compat.v1.zeros([784, 10]))
b = tf.compat.v1.Variable(tf.compat.v1.zeros([10]))

with tf.compat.v1.name_scope("Wx_b") as scope:
    # Construct a linear model
    model = tf.compat.v1.nn.softmax(tf.matmul(x, W) + b) # Softmax
    
# Add summary ops to collect data
w_h = tf.compat.v1.summary.histogram("weights", W)
b_h = tf.compat.v1.summary.histogram("biases", b)

In [14]:
# More name scopes will clean up graph representation
with tf.compat.v1.name_scope("cost_function") as scope:
    # Minimize error using cross entropy
    # Cross entropy
    cost_function = -tf.compat.v1.reduce_sum(y*tf.compat.v1.log(model))
    # Create a summary to monitor the cost function
    tf.compat.v1.summary.scalar("cost_function", cost_function)

In [15]:
with tf.compat.v1.name_scope("train") as scope:
    # Gradient descent
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [16]:
# Initializing the variables
init = tf.compat.v1.global_variables_initializer()

# Merge all summaries into a single operator
merged_summary_op = tf.compat.v1.summary.merge_all()

# Launch the graph
with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    # Set the logs writer to the folder /tmp/tensorflow_logs
    summary_writer = tf.compat.v1.summary.FileWriter('logs2/', graph=sess.graph)

    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Tuning completed!")
    # Test the model
    predictions = tf.compat.v1.equal(tf.compat.v1.argmax(model, 1), tf.compat.v1.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.compat.v1.reduce_mean(tf.compat.v1.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Iteration: 0001 cost= 30.843153863
Iteration: 0003 cost= 21.052430659
Iteration: 0005 cost= 20.168268670
Iteration: 0007 cost= 19.644195030
Iteration: 0009 cost= 19.462901025
Iteration: 0011 cost= 19.152972166
Iteration: 0013 cost= 18.952092522
Iteration: 0015 cost= 18.813775002
Iteration: 0017 cost= 18.586912472
Iteration: 0019 cost= 18.514569128
Iteration: 0021 cost= 18.519927443
Iteration: 0023 cost= 18.277574659
Iteration: 0025 cost= 18.312340253
Iteration: 0027 cost= 18.152031144
Iteration: 0029 cost= 18.043431007
Tuning completed!
Accuracy: 0.9182


In [17]:
#Launching TensorBoard
#To run TensorBoard, use the following command (alternatively python -m tensorboard.main)

#tensorboard --logdir=path/to/log-directory

In [18]:
#Once TensorBoard is running, navigate your web browser to localhost:6006 to view the TensorBoard.